## 事前準備

### モジュールのインポート

In [1]:
import warnings
warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')

In [2]:
# 自作モジュール
from utils.paths import Paths
from facades.stock_acquisition_facade import StockAcquisitionFacade
from utils.jquants_api_utils import cli
from calculation.target_calculator import TargetCalculator
# 基本モジュール
from datetime import datetime
import pandas as pd
import numpy as np
import os
# クラスタリングで使用
from sklearn.decomposition import PCA
import umap
from pyclustering.cluster.xmeans import xmeans, splitting_type
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from sklearn.cluster import KMeans
from hdbscan import HDBSCAN
from scipy.spatial.distance import cdist

# 結果の描画に使用
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## データセットの準備

### 銘柄情報の取得
* stock_lists: 2014年10月からの銘柄一覧
* history_list: 銘柄ごとのScaleCategoryの遍歴

In [ ]:
filter_condition = "(Listing==1)&((ScaleCategory=='TOPIX Core30')|(ScaleCategory=='TOPIX Large70')|(ScaleCategory=='TOPIX Mid400')|(ScaleCategory=='TOPIX Small 1'))" #現行のTOPIX500""
saf = StockAcquisitionFacade(filter=filter_condition) #filtered_code_list = filter_codes)
stock_dfs = saf.get_stock_data_dict()
stock_dfs['list']

### 時系列データの成型

In [ ]:
# 目的変数（日内リターン）を算出
stock_dfs['price']['Target'] = stock_dfs['price']['Close'] / stock_dfs['price']['Open'] - 1
target = stock_dfs['price'][['Date', 'Code', 'Target']]
target = target.set_index(['Date', 'Code'], drop=True).unstack(-1).droplevel(0, axis=1)
target

## 関数群

### PCA処理の実行

In [5]:
def extract_pca_residues(target: pd.DataFrame, remove_components: int, end_date: datetime = datetime(2021, 12, 31)):
    '''
    remove_componentsで指定した数までの主成分を除去して、残差を返します。
    '''
    target = target[target.index <= end_date]
    no_missing_target = target.dropna(axis=1).T
    if remove_components == 0:
        return no_missing_target
    pca = PCA(n_components = remove_components).fit(no_missing_target)
    pca_array = pca.transform(no_missing_target)
    inversed_array = pca.inverse_transform(pca_array)

    return no_missing_target - inversed_array

### UMAPの適用

In [6]:
def apply_UMAP(df:pd.DataFrame, n_components:int=15, n_neighbors:int=5, min_dist:float=0.1, metric:str='euclidean'):
    UMAP_model = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, metric=metric, random_state=42)
    UMAP_result = UMAP_model.fit_transform(df)
    reduced_df = pd.DataFrame(UMAP_result, index=df.index, columns=['Feature '+ str(i) for i in range(0, n_components)])
    reduced_df = reduced_df.sort_index(ascending=True).reset_index(drop=False)
    return reduced_df

### 社名の追加

In [7]:
def add_company_name(df, config_df):
    return pd.merge(config_df[['Code', 'CompanyName']], df, on='Code').set_index(['Code', 'CompanyName'])

### xmeansによるクラスタリング

In [8]:
def apply_xmeans(df, kmax: int, amount_centers: str = 2, tolerrance:float = 0.001):
    xm_c = kmeans_plusplus_initializer(data=df, amount_centers=2).initialize()
    xm_i = xmeans(data=df, initial_centers=xm_c, kmax=kmax, tolerrance=0.001, ccore=True)
    xm_i.process()

    centers = xm_i.get_centers()
    df['Cluster'] = 999
    df['ClusterCenter0'] = 0.0
    df['ClusterCenter1'] = 0.0
    for cluster_num, cluster in enumerate(xm_i.get_clusters()):
        df.iloc[cluster, df.columns.get_loc('Cluster')] = cluster_num
        df.iloc[cluster, df.columns.get_loc('ClusterCenter0')] = centers[cluster_num][0]
        df.iloc[cluster, df.columns.get_loc('ClusterCenter1')] = centers[cluster_num][1]
    
    df['DistanceFromClusterCenter'] = (
        (df['Feature 0'] - df['ClusterCenter0']) ** 2 + \
        (df['Feature 1'] - df['ClusterCenter1']) ** 2
        ) ** 0.5


        
    return df[['Feature 0', 'Feature 1', 'Cluster', 'DistanceFromClusterCenter']]

In [9]:
def apply_hdbscan(df, min_cluster_size: int = 10, min_samples: int = 5, *args, **kwargs):
    hdbscan = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, *args, **kwargs)
    df['Cluster'] = hdbscan.fit_predict(df)
        
    return df

### 結果のプロット

In [10]:
def show_in_plotly(df, n_clusters=5):
    """
    マルチインデックス（銘柄コードと社名）を持つUMAPデータをクラスタリングして可視化する
    
    Parameters:
    -----------
    df : pandas.DataFrame
        UMAPの結果を含むデータフレーム（マルチインデックス[Code, CompanyName]付き）
    n_clusters : int
        クラスター数
    """
    import plotly.express as px
    import plotly.graph_objects as go
    from sklearn.cluster import KMeans
    import pandas as pd
    
    # インデックスから情報を取得
    df_plot = df.copy()
    
    # マルチインデックスをリセットして通常の列に変換
    df_plot = df_plot.reset_index()
    
    # 散布図の作成
    fig = px.scatter(
        df_plot,
        x='Feature 0',
        y='Feature 1',
        color='Cluster',
        color_discrete_sequence=px.colors.qualitative.Bold,
        hover_data={
            'Code': True,
            'CompanyName': True,
            'Cluster': True,
            'Feature 0': ':.4f',
            'Feature 1': ':.4f'
        },
        title='UMAP次元削減による株価データの可視化',
        labels={'Feature 0': 'UMAP次元1', 'Feature 1': 'UMAP次元2', 'Cluster': 'クラスター'},
        opacity=0.7
    )
 
    
    # ホバーテンプレートをカスタマイズ（銘柄コード、社名、クラスターを表示）
    fig.update_traces(
        selector=dict(type='scatter', mode='markers'),
        hovertemplate='<b>銘柄コード: %{customdata[0]}</b><br><b>社名: %{customdata[1]}</b><br><b>クラスター: %{customdata[2]}</b><br>UMAP次元1: %{x:.4f}<br>UMAP次元2: %{y:.4f}<extra></extra>'
    )
    
    # レイアウトの調整
    fig.update_layout(
        width=900,
        height=700,
        plot_bgcolor='white',
        legend_title_text='クラスター',
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            zeroline=True,
            zerolinewidth=1,
            zerolinecolor='lightgray',
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            zeroline=True,
            zerolinewidth=1,
            zerolinecolor='lightgray',
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
        )
    )
    
    # 図を表示
    fig.show()
    
    # クラスタリング結果を含むデータフレームを返す
    # マルチインデックスに戻す
    df_result = df_plot.set_index(['Code', 'CompanyName'])
    
    return fig, df_result

## 実行

In [ ]:
codes =["5310","4047","3101","4045","4403","4021","4272","5901","4028","4044","4095","4046","4212","4617","4008","4023","4041","4996","4634","4216","4078","4633","4401","4109","4471","4114","4088","4091","8012","8098","3401","3405","4203","3402","3407","4205","4631","4043","4042","4182","4208","4183","4005","4188","4061","4118","4202"]

target2 = target.loc[:, codes]
target2.tail(2)

In [134]:
dfs = []
remove_pc = 0
target2 = target
target2 = target2[target2.index <= datetime(2021, 12, 31)]
no_missing_target = target2.dropna(axis=1)
no_missing_target = no_missing_target


In [ ]:
corr_means = []

for reduce_components in range(len(no_missing_target) - 1):
    residual_df = extract_pca_residues(no_missing_target, reduce_components).T
    corr_mean = ((residual_df.corr().apply(abs).sum() - 1) / len(residual_df.columns)).mean()
    print(f'{reduce_components}: {corr_mean}')
    corr_means.append(corr_mean)


In [ ]:
dfs = []
remove_pc = 0
#target2 = target
target2 = target2[target2.index <= datetime(2021, 12, 31)]
no_missing_target = target2.dropna(axis=1).T

#for pca_components in range(2, len(no_missing_target.index)):
for pca_components in range(10, 26):
    pca = PCA(n_components = pca_components).fit(no_missing_target)
    pca_array = pca.transform(no_missing_target)
    inversed_array = pca.inverse_transform(pca_array)
    print(f'累積寄与率：{sum(pca.explained_variance_ratio_)}')
    target_pcaed = pd.DataFrame(inversed_array, index=no_missing_target.index, 
                        columns=pd.to_datetime(no_missing_target.columns)).T

    residual_df = extract_pca_residues(target_pcaed, remove_pc)
    umaped_df = apply_UMAP(residual_df, n_components=2, n_neighbors=2, min_dist=0.01, metric='correlation')
    df = add_company_name(umaped_df, stock_dfs['list'])
    df = apply_hdbscan(df, min_cluster_size=2, min_samples=1, cluster_selection_epsilon = 0.7)
    df = df[['Cluster']].rename(columns={'Cluster': f'Cluster{pca_components}'})
    dfs.append(df)

df = pd.concat(dfs, axis=1)

def compare_rows(df):
    """
    データフレームの各行を互いに比較し、一致する値の数をカウントする新しいデータフレームを返す
    
    Parameters:
    -----------
    df : pandas.DataFrame
        入力データフレーム
    code_col : str, default='Code'
        コード列の名前
        
    Returns:
    --------
    pandas.DataFrame
        各行のペア間で一致する値の数を示す新しいデータフレーム
    """
    
    # 結果を格納するための空のデータフレームを作成
    result = pd.DataFrame(index=df.index, columns=df.index)
    
    # 各行のペアを比較
    for idx1 in df.index:
        for idx2 in df.index:
            # 同じ値を持つ列の数をカウント
            matches = (df.loc[idx1] == df.loc[idx2]).sum()
            result.loc[idx1, idx2] = matches
    
    return result #- np.eye(len(result.index)) * (len(result.index) - 1)


result = compare_rows(df)
df.to_csv(f'Clusters.csv')
result.to_csv(f'CompareResult.csv')

In [ ]:
pca_components = 10
remove_pc = 0

#target2 = target
target2 = target2[target2.index <= datetime(2021, 12, 31)]
no_missing_target = target2.dropna(axis=1).T
pca = PCA(n_components = pca_components).fit(no_missing_target)
pca_array = pca.transform(no_missing_target)
inversed_array = pca.inverse_transform(pca_array)
print(f'累積寄与率：{sum(pca.explained_variance_ratio_)}')
target_pcaed = pd.DataFrame(inversed_array, index=no_missing_target.index, 
                      columns=pd.to_datetime(no_missing_target.columns)).T

residual_df = extract_pca_residues(target_pcaed, remove_pc)
umaped_df = apply_UMAP(residual_df, n_components=2, n_neighbors=2, min_dist=0.01, metric='correlation')
df = add_company_name(umaped_df, stock_dfs['list'])
df = apply_hdbscan(df, min_cluster_size=2, min_samples=1, cluster_selection_epsilon = 0.7)
print(f'除去する主成分数：{remove_pc}')
show_in_plotly(df)
df.to_csv(f'ClustersRemove{remove_pc}PCs.csv')

## セクター判定

In [ ]:
for remove_pc in range(0, 10):
    residual_df = extract_pca_residues(target2, remove_pc)
    umaped_df = apply_UMAP(residual_df, n_components=2, n_neighbors = 3, min_dist=0, metric='cosine')
    df = add_company_name(umaped_df, stock_dfs['list'])
    df = apply_xmeans(df, kmax=50)
    print(f'除去する主成分数：{remove_pc}')
    show_in_plotly(df)
    df.to_csv(f'ClustersRemove{remove_pc}PCs.csv')

In [ ]:
for remove_pc in range(0, 10):
    residual_df = extract_pca_residues(target2, remove_pc)
    umaped_df = apply_UMAP(residual_df, n_components=2, n_neighbors = 3, min_dist=0, metric='cosine')
    df = add_company_name(umaped_df, stock_dfs['list'])
    df = apply_xmeans(df, kmax=50)
    print(f'除去する主成分数：{remove_pc}')
    show_in_plotly(df)
    df.to_csv(f'ClustersRemove{remove_pc}PCs.csv')

## 検討

### サブプロットを作成する関数

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

def show_umap_subplots(df_dict, n_neighbors_list, min_dist_list):
    """
    複数のUMAPパラメータ設定による結果を4×4のサブプロットとして可視化
    
    Parameters:
    -----------
    df_dict : dict
        キー: (n_neighbors, min_dist) のタプル
        値: 対応するパラメータでのUMAP結果のデータフレーム
    n_neighbors_list : list
        n_neighborsの値のリスト [3, 5, 10, 15]
    min_dist_list : list
        min_distの値のリスト [0, 0.01, 0.05, 0.1]
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        プロットされた図
    """
    # サブプロットタイトル
    subplot_titles = [f'n_neighbors={n}, min_dist={d}' 
                     for n in n_neighbors_list 
                     for d in min_dist_list]
    
    # サブプロットの作成
    fig = make_subplots(
        rows=4, cols=4,
        subplot_titles=subplot_titles,
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )
    
    # カラーシーケンスの取得
    color_sequence = px.colors.qualitative.Bold
    
    # 各パラメータ組み合わせのプロットを追加
    for i, n_neighbors in enumerate(n_neighbors_list):
        for j, min_dist in enumerate(min_dist_list):
            # 該当するパラメータ組み合わせのデータフレームを取得
            key = (n_neighbors, min_dist)
            if key not in df_dict:
                continue
                
            df_plot = df_dict[key].copy().reset_index()
            
            # ユニーククラスターを取得
            clusters = df_plot['Cluster'].unique()
            
            # 各クラスターを別々のトレースとして追加
            for cluster_id in sorted(clusters):
                cluster_df = df_plot[df_plot['Cluster'] == cluster_id]
                color_idx = cluster_id % len(color_sequence)
                
                fig.add_trace(
                    go.Scatter(
                        x=cluster_df['Feature 0'],
                        y=cluster_df['Feature 1'],
                        mode='markers',
                        marker=dict(
                            size=6,
                            color=color_sequence[color_idx],
                            opacity=0.7,
                            line=dict(width=0.5, color='white')
                        ),
                        name=f'Cluster {cluster_id}',
                        legendgroup=f'Cluster {cluster_id}',
                        showlegend=False,  # サブプロットが多いため凡例は非表示
                        customdata=np.column_stack((
                            cluster_df['Code'],
                            cluster_df['CompanyName'],
                            cluster_df['Cluster']
                        )),
                        hovertemplate=(
                            '<b>銘柄コード: %{customdata[0]}</b><br>' +
                            '<b>社名: %{customdata[1]}</b><br>' +
                            '<b>クラスター: %{customdata[2]}</b><br>' +
                            'UMAP次元1: %{x:.4f}<br>' +
                            'UMAP次元2: %{y:.4f}<br>' +
                            '<extra></extra>'
                        )
                    ),
                    row=i+1, col=j+1
                )
            
            # クラスタ数をアノテーションとして追加
            cluster_count = len(clusters)
            fig.add_annotation(
                x=0.5, y=0.02,
                text=f'クラスター数: {cluster_count}',
                showarrow=False,
                xref=f'x{i*4+j+1}', yref=f'y{i*4+j+1}',
                font=dict(size=10, color="black"),
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="black",
                borderwidth=1,
                borderpad=4,
                row=i+1, col=j+1
            )
    
    # レイアウト設定
    fig.update_layout(
        title='UMAP パラメータ比較 (4×4 サブプロット)',
        height=900,
        width=1200,
        template="plotly_white",
        margin=dict(l=20, r=20, t=50, b=20),
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
        )
    )
    
    # 軸のレイアウト設定
    fig.update_xaxes(
        showticklabels=False,
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='lightgray',
        gridwidth=1,
        gridcolor='lightgray',
        title_text=''
    )
    
    fig.update_yaxes(
        showticklabels=False,
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='lightgray',
        gridwidth=1,
        gridcolor='lightgray',
        title_text=''
    )
    
    return fig

# 使用例
"""
# 例えば以下のようにして使用します:
n_neighbors_list = [3, 5, 10, 15]
min_dist_list = [0, 0.01, 0.05, 0.1]

# 結果を格納する辞書
df_dict = {}

# 各パラメータ組み合わせの処理
for n_neighbors in n_neighbors_list:
    for min_dist in min_dist_list:
        # UMAPとクラスタリングを実行
        residual_df = extract_pca_residues(target, 0)
        umaped_df = apply_UMAP(residual_df, n_components=2, n_neighbors=n_neighbors, min_dist=min_dist, metric='correlation')
        df = add_company_name(umaped_df, stock_dfs['list'])
        df = apply_xmeans(df, kmax=50)
        
        # 結果を辞書に保存
        df_dict[(n_neighbors, min_dist)] = df

# サブプロットの可視化
fig = show_umap_subplots(df_dict, n_neighbors_list, min_dist_list)
fig.show()

# 保存する場合
# fig.write_html("umap_parameter_comparison.html")
"""

### n_neighborsとmin_distの検討

In [ ]:
n_neighbors_list = [2, 3, 5, 10]
min_dist_list = [0.001, 0.1, 0.3, 0.5]
df_dict = {}
df_list_for_csv = []


target = target[target.index <= datetime(2021, 12, 31)]
no_missing_target = target.dropna(axis=1).T
pca = PCA(n_components = 150).fit(no_missing_target)
pca_array = pca.transform(no_missing_target)
inversed_array = pca.inverse_transform(pca_array)
print(f'累積寄与率：{sum(pca.explained_variance_ratio_)}')
target_pcaed = pd.DataFrame(inversed_array, index=no_missing_target.index, 
                      columns=pd.to_datetime(no_missing_target.columns)).T

for n_neighbors in n_neighbors_list:
    for min_dist in min_dist_list:
        residual_df = extract_pca_residues(target_pcaed, 0)
        umaped_df = apply_UMAP(residual_df, n_components=2, n_neighbors = n_neighbors, min_dist=min_dist, metric='correlation')
        df = add_company_name(umaped_df, stock_dfs['list'])
        df = apply_hdbscan(df, min_cluster_size=2, min_samples=1, cluster_selection_epsilon = 0.3)
        df_dict[(n_neighbors, min_dist)] = df
        df_list_for_csv.append(df[['Cluster']].rename(columns={'Cluster': f'{n_neighbors}nbs_{min_dist}_md'}))
        
final_df = pd.concat(df_list_for_csv, axis=1)
final_df.to_csv('Clusters_UmapParamExploring.csv')

fig = show_umap_subplots(df_dict, n_neighbors_list, min_dist_list)
fig.show()

### umapのパラメータ検討

#### n_neighbors（小さいほど局所に注目）

In [ ]:
for n_neighbors in [5, 15, 25, 35, 45]:
    residual_df = extract_pca_residues(target, 1)
    umaped_df = apply_UMAP(residual_df, n_components=2, n_neighbors = n_neighbors, metric='euclidean')
    df = add_company_name(umaped_df, stock_dfs['list'])
    df = apply_xmeans(df, kmax=5)
    print(f'n_neighbors: {n_neighbors}')
    show_in_plotly(df)

#### min_dist（小さいほどクラスタの凝集度が高まる）

In [ ]:
for min_dist in [0, 0.2, 0.4, 0.6, 0.8, 1]:
    residual_df = extract_pca_residues(target, 1)
    umaped_df = apply_UMAP(residual_df, n_components=2, min_dist = min_dist, n_neighbors = 5, metric='euclidean')
    df = add_company_name(umaped_df, stock_dfs['list'])
    df = apply_xmeans(df, kmax=5)
    print(f'min_dist: {min_dist}')
    show_in_plotly(df)

#### metric ("euclidean": ユークリッド距離、"cosine": ベクトル情報を使用)

#### 各metricの簡易説明

 1. euclidean（ユークリッド距離）:
    - 最も基本的な直線距離。
    - 連続値データに対して直感的でよく使われるが、ノイズにやや敏感。

 2. manhattan（マンハッタン距離）:
    - 各次元の絶対差の合計（碁盤の目の移動距離）。
    - 高次元や外れ値に強く、頑健な評価をしたい場合に有効。

 3. chebyshev（チェビシェフ距離）:
    - 各次元の差の中で最大のものを距離とする。
    - 一部の次元で大きな違いがあるときに顕著に効く。

 4. minkowski（ミンコフスキー距離）:
    - ユークリッドやマンハッタンを含む一般化された距離。
    - パラメータpにより距離の性質を調整できる（p=1→マンハッタン, p=2→ユークリッド）。

 5. hamming（ハミング距離）:
    - 同じ長さのベクトルに対して、異なる位置の数をカウント。
    - 主にバイナリデータやカテゴリデータの比較に使用。

 6. cosine（コサイン距離）:
    - ベクトルのなす角に基づいた距離（1 - 類似度）。
    - 長さではなく「向き」に注目。高次元で疎なデータ（例：テキスト）に強い。

#### 実行コード

In [ ]:
for metric in ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'hamming', 'cosine']:
    residual_df = extract_pca_residues(target, 1)
    umaped_df = apply_UMAP(residual_df, n_components=2, min_dist = 0, n_neighbors = 5, metric=metric)
    df = add_company_name(umaped_df, stock_dfs['list'])
    df = apply_xmeans(df, kmax=50, tolerrance=0.01)
    print(f'metric: {metric}')
    show_in_plotly(df)